## Building A Chatbot
In this lecture We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This lecture tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A8CC246C80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A8CC27C0A0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage("Hello, My name is Daveworld and i am an AI Enginner")])

AIMessage(content="Hello Daveworld! It's nice to meet you.  \n\nThat's a fascinating field! What kind of AI engineering do you specialize in? Are you working on anything particularly exciting at the moment? \n\nI'm always eager to learn more about what my fellow AI enthusiasts are up to. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 23, 'total_tokens': 91, 'completion_time': 0.123636364, 'prompt_time': 0.002118105, 'queue_time': 0.019962524000000002, 'total_time': 0.125754469}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9fc435a8-52e6-4112-9cb7-646b99f131a7-0', usage_metadata={'input_tokens': 23, 'output_tokens': 68, 'total_tokens': 91})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hello, My name is Daveworld and i am an AI Enginner"),
        AIMessage(content="Hello Daveworld! It's nice to meet you.  \n\nThat's a fascinating field! What kind of AI engineering do you specialize in? Are you working on anything particularly exciting at the moment? \n\nI'm always eager to learn more about what my fellow AI enthusiasts are up to. 😊  \n\n"),
        HumanMessage(content="WHat is my name")
    ]
)

AIMessage(content='Your name is Daveworld.  You told me at the beginning of our conversation! 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 103, 'total_tokens': 125, 'completion_time': 0.04, 'prompt_time': 0.005502608, 'queue_time': 0.018772242, 'total_time': 0.045502608}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0e66ba25-f77c-4522-9dbd-9e4a4834aef3-0', usage_metadata={'input_tokens': 103, 'output_tokens': 22, 'total_tokens': 125})

In [6]:
model.invoke([HumanMessage("WHat us my name?")])

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 15, 'total_tokens': 55, 'completion_time': 0.072727273, 'prompt_time': 0.001898276, 'queue_time': 0.019722582, 'total_time': 0.074625549}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-615da675-f975-4a1f-90d1-12f9e83c3695-0', usage_metadata={'input_tokens': 15, 'output_tokens': 40, 'total_tokens': 55})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
!pip install langchain_community

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [19]:
store = {} # This is a dictionary that will store the chat history

# create a function to get session history
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store: # checkig if the session history exists
        store[session_id] = ChatMessageHistory() #creating a new session history if it does not exist
    return store[session_id] # returning the session history

In [20]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [11]:
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x000001A8CC25A950>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [21]:
config = {"configurable": {"session_id": "chat1"}}

In [22]:
config

{'configurable': {'session_id': 'chat1'}}

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, My name is Daveworld and i am an AI Enginner")],
    config=config
)

In [24]:
response.content

"Hello Daveworld!  \n\nIt's nice to meet you. That's a great field to be in - AI engineering is so exciting right now with all the advancements happening.  \n\nWhat kind of AI projects are you working on?  \n\n"

In [25]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content='According to our conversation, your name is Daveworld. 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 89, 'total_tokens': 116, 'completion_time': 0.049090909, 'prompt_time': 0.005494972, 'queue_time': 0.017199646, 'total_time': 0.054585881}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-99c8c79c-d3a5-4567-a6f9-2e9480a61815-0', usage_metadata={'input_tokens': 89, 'output_tokens': 27, 'total_tokens': 116})

In [26]:
# Creating a new session
config2 = {"configurable": {"session_id": "chat2"}}
response2 = with_message_history.invoke(
    [HumanMessage(content="WHat is my name?")],
    config=config2
)

In [27]:
response2.content

"As an AI, I have no access to your personal information, including your name. If you'd like to tell me your name, I'd be happy to use it!\n"

In [28]:
response2 = with_message_history.invoke(
    [HumanMessage(content="Hey, my name is Dave")],
    config=config2
)
response2.content

'Hi Dave! Nice to meet you.  What can I do for you today? 😊  \n\n'

In [29]:
response2 = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config2
)
response2.content

'Your name is Dave!  \n\nI remember that you told me earlier. 😊  Anything else I can help you with, Dave? \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides the messages.

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="user_message"), # This is a placeholder for the user message
    ]
)

chain = prompt | model

In [31]:
chain.invoke({"user_message": [HumanMessage(content="Hi my name is Daveworld")]})

AIMessage(content="Hi Daveworld, it's nice to meet you!  \n\nWhat can I do for you today? I'm ready for your questions. 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 31, 'total_tokens': 66, 'completion_time': 0.063636364, 'prompt_time': 0.002358226, 'queue_time': 0.018416823, 'total_time': 0.06599459}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-053d0e20-b2ec-4e14-a170-9d54ed8d04c0-0', usage_metadata={'input_tokens': 31, 'output_tokens': 35, 'total_tokens': 66})

In [32]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [33]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, My name is Daveworld and i am an AI Enginner', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Daveworld!  \n\nIt's nice to meet you. That's a great field to be in - AI engineering is so exciting right now with all the advancements happening.  \n\nWhat kind of AI projects are you working on?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 23, 'total_tokens': 77, 'completion_time': 0.098181818, 'prompt_time': 0.002112527, 'queue_time': 0.016745213000000002, 'total_time': 0.100294345}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-59a44980-644a-4edd-bcd9-f758369b6ef0-0', usage_metadata={'input_tokens': 23, 'output_tokens': 54, 'total_tokens': 77}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Accordin

In [34]:
config3 = {"configurable": {"session_id": "chat3"}}
response3 = with_message_history.invoke(
    [HumanMessage(content="Hi my name is Daveworld")],
    config=config3
)

response3.content

"Hello Daveworld! It's nice to meet you.  \n\nI'm ready to answer your questions. What can I do for you today? 😁  \n\n"

In [83]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in this language:{language}"),
        MessagesPlaceholder(variable_name="user_message"), # This is a placeholder for the user message
    ]
)

chain = prompt | model

In [84]:
response4 = chain.invoke({"user_message": [HumanMessage(content="Hi my name is Daveworld")], "language": "yoruba"})
response4.content

"Ẹ kú àárọ̀, Daveworld!  \n\nMi o le ṣe ìrànwọ́ ènìyàn. Ẹ́yàn mọ́ wọ́n ni ń lọ́wọ́ mi. \n\n\n(Hello, Daveworld!\n\nI can't do human things. People are helping me.) \n\nO dábọ̀ ọ̀yà kan fún mi?\n\n\n\n(Do you have a question for me?) \n\n"

Let's go ahead wrap this in more complicated chains in a message history class.

In [78]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="user_message", # This is the key for the input message
)

In [79]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, My name is Daveworld and i am an AI Enginner', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Daveworld!  \n\nIt's nice to meet you. That's a great field to be in - AI engineering is so exciting right now with all the advancements happening.  \n\nWhat kind of AI projects are you working on?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 23, 'total_tokens': 77, 'completion_time': 0.098181818, 'prompt_time': 0.002112527, 'queue_time': 0.016745213000000002, 'total_time': 0.100294345}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-59a44980-644a-4edd-bcd9-f758369b6ef0-0', usage_metadata={'input_tokens': 23, 'output_tokens': 54, 'total_tokens': 77}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Accordin

In [88]:
config4 = {"configurable": {"session_id": "chat5"}}
response4 = with_message_history.invoke({"user_message": [HumanMessage(content="Hi my name is Daveworld")], "language": "igbo"},
    config=config4
)

response4.content

'Ehee! Daveworld, ka unu bu. Ụrụ ị ga-achịkwa m? \n\n(What can I do for you?)\n'

In [92]:
reponse5 = with_message_history.invoke(
    {"user_message": [HumanMessage(content="What is my name")], "language": "igbo"},
    config=config4
)

In [93]:
reponse5.content

'Aha gị bụ Daveworld.  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [108]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=30, ## the maximum number of tokens
    strategy="last", ## the strategy to use (last or first) last focus on the last conversation it got
    token_counter=model, ## the token counter
    include_system=True, ## include the system message
    allow_partial=False, ## allow partial messages
    start_on="human" ## start on the human message
)

messages_new = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages_new)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [111]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("user_message") | trimmer) 
    | prompt
    | model
)

response7 = chain.invoke(
    {
        "user_message": messages_new  + [HumanMessage(content="What is my name")],
        "language": "english"    
    }
)

In [112]:
response7.content

'Your name is Bob! 😄\n'

In [105]:
response6 = chain.invoke(
    {
        "user_message": messages  + [HumanMessage(content="What type of ice cream do i like?")],
        "language": "english"    
    }
)

In [106]:
response6.content

'You said you like vanilla ice cream. 😊  \n\n\n\n'

In [114]:
response = with_message_history.invoke(
    {
        "user_message" : messages_new + [HumanMessage(content="What is my name")],
        "language": "English"
    },
    config=config
)

In [115]:
response.content

'Your name is Bob!  \n\nIs there anything else I can help you with?'